In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import sys
from  fractions import Fraction
import timeit

In [ ]:
def generate_synthetic_graph(a,b,c,deg_in,deg_out,h,n,p): #Anwar2020 homophily attachment model
    G = nx.DiGraph()
    nx.set_node_attributes(G, [], "community")
    nodes=0
    for i in range(n):
        if random.uniform(0, 1)<=a:
            G.add_node(nodes)
            G.nodes[nodes]['community']= 'majority' if random.uniform(0, 1)<=p else 'minority'
            if nodes>0: # we need at least another node to connect to
                weights=[]
                for node in range(nodes): #so that we dont create a self loop we dont consider the node we just added
                    homophily= h*G.in_degree(node)+deg_in if G.nodes[nodes]['community']==G.nodes[node]['community'] else (1-h)*G.in_degree(node)+deg_in
                    weights.append(homophily)

                w= random.choices(range(nodes), weights=weights, k=1)[0]
                G.add_edge(nodes, w)
            nodes+=1
            
        if random.uniform(0, 1)<=b and nodes>2:
            v= random.choices(range(nodes), weights=[G.out_degree(node)+deg_out for node in G.nodes() ], k=1)[0]
            weights=[]
            for node in G.nodes():
                homophily= h*G.in_degree(node)+deg_in if G.nodes[v]['community']==G.nodes[node]['community'] else (1-h)*G.in_degree(node)+deg_in
                weights.append(homophily)
            options=list(range(nodes))
            options.remove(v)
            w= random.choices(options, weights=weights.remove(weights[v]), k=1)[0]
            G.add_edge(v, w)
            
        if random.uniform(0, 1)<=c:
            G.add_node(nodes)
            G.nodes[nodes]['community']= 'majority' if random.uniform(0, 1)<=p else 'minority'
            if nodes>0: # we need at least another node to connect to
                weights=[]
                for node in range(nodes): #so that we dont create a self loop
                    homophily= h*G.out_degree(node)+deg_out if G.nodes[nodes]['community']==G.nodes[node]['community'] else (1-h)*G.out_degree(node)+deg_out
                    weights.append(homophily)
                w= random.choices(range(nodes), weights=weights, k=1)[0]
                G.add_edge(w, nodes)
            nodes+=1
            
    return G 
size=80
start = timeit.default_timer()
Graph=generate_synthetic_graph(1/4,1/2,1/4,1,1,0.5,size,0.7)
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(len(Graph.nodes()))
if len(Graph.nodes())< 50:
    pos = nx.kamada_kawai_layout(Graph)
    nx.draw(Graph,pos,node_color=['green' if Graph.nodes[v]['community']== 'majority' else 'red' for v in Graph.nodes()],with_labels=1) 

In [ ]:
def compute_MIA(v, G, in_edges, min_prob): # use dijstra to compute it
    MIA = nx.DiGraph()
    unvisited_nodes = list(G.nodes())
    shortest_path = {}
    previous_nodes = {}
    probabilities={}
    
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
        probabilities[node]=0
    shortest_path[v] = 0
    probabilities[v] = 1
    MIA.add_node(v)
    
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes: 
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node

        if not in_edges:
            neighbors = G.successors(current_min_node)
        else:
            neighbors = G.predecessors(current_min_node)    
        for neighbour in neighbors:
            if in_edges:
                prob=-np.log(1/G.in_degree(current_min_node))
            else:
                prob=-np.log(1/G.in_degree(neighbour))
                
            tentative_value = shortest_path[current_min_node] +prob
            if tentative_value < shortest_path[neighbour]:
                shortest_path[neighbour] = tentative_value 
                previous_nodes[neighbour] = current_min_node
        
        unvisited_nodes.remove(current_min_node)
        
    for w in  previous_nodes.keys(): 
        if  in_edges:
            if probabilities[w] !=0:
                probabilities[previous_nodes[w]]= 1/G.in_degree(previous_nodes[w])*probabilities[w]
            else:
                probabilities[previous_nodes[w]]= 1/G.in_degree(previous_nodes[w])
            MIA.add_edge(w,previous_nodes[w])
        else:
            if probabilities[previous_nodes[w]]!=0:
                probabilities[w]= 1/G.in_degree(w)*probabilities[previous_nodes[w]]
            else:
                probabilities[w]= 1/G.in_degree(w)
            MIA.add_edge(previous_nodes[w],w)
            
    subset_nodes=[v for v,l in probabilities.items() if l < min_prob and v in previous_nodes.keys()]
    for i in subset_nodes:
        MIA.remove_nodes_from([i])
        shortest_path.pop(i)
        previous_nodes.pop(i) 
    return MIA,shortest_path,previous_nodes
    

In [ ]:
def compute_PIIS(u,G,d):  
    visited=[]
    PIIS=[]
    queue=[]
    visited.append(u)
    queue.append((u,0))
    while queue:
        s = queue.pop(0)
        PIIS.append(s[0])
        list_neigh=[]
        list_neig=G.predecessors(s[0])
        for neighbour in list_neig:
            if neighbour not in visited and s[1] < d :
                visited.append(neighbour)
                queue.append((neighbour,s[1]+1))
    return PIIS

In [ ]:
def compute_PIOS(u,G,D_max): 
    visited=[]
    PIOS=[]
    queue=[]
    visited.append(u)
    queue.append((u,0))
    while queue:
        s = queue.pop(0)
        PIOS.append(s[0])
        list_neigh=[]
        list_neig=G.successors(s[0])
        for neighbour in list_neig:
            if neighbour not in visited and s[1] < D_max[neighbour]:
                visited.append(neighbour)
                queue.append((neighbour,s[1]+1))
    return PIOS

In [ ]:
def compute_ap_N(v,S_N,MIA,G): #probability of transmission is inversly proportional to the in-degree 
    if v in S_N: 
        ap=Fraction(1,1)
    elif not list(MIA.predecessors(v)): 
        ap=Fraction(0,1)
    else:
        prob=Fraction(1,1) 
        for u in MIA.predecessors(v): 
            prob*=Fraction(1,1)-(compute_ap_N(u,S_N,MIA,G)*Fraction(1,G.in_degree(v)))
        ap=Fraction(1,1)-prob
    return ap

In [ ]:
def find_dc(G,S_P,v):
    visited=[]
    queue=[] 
    queue.append((v,0))
    visited.append(v)
    while queue:
        s = queue.pop(0) 
        if s[0] in S_P:
            return s[1]
        for neighbour in G.successors(s[0]):
            if neighbour not in visited:
                if neighbour in S_P:
                    return s[1]+1
                visited.append(neighbour)
                queue.append((neighbour,s[1]+1))
    return sys.maxsize

In [ ]:
def CMIA_H(G,S_N,k,min_prob): # for this algo we assume the prob of possitive diffusion is 1
    S_P=[]
    Neg_range=set()
    DecInf={}
    dist={}
    D={} # dict to keep track of the distances values per node, and avoid reconstructing MIA
    D_max=[0]*len(G.nodes()) # dict to keep track of the dmax values per node
    MIA_prev_nodes={}# to keep track of the MIIAs without having to reconstruct
    
    start = timeit.default_timer()

    for u in S_N:
        MIOA,_,_=compute_MIA(u, G, False, min_prob)
        for v in list(set(MIOA.nodes()) - set(S_N)):
            Neg_range.add(v) # contains the nodes that could get infected
    print(len(Neg_range))
    for u in Neg_range: 
        MIIA,shortest_path,previous_nodes=compute_MIA(u, G, True, min_prob)
        
        dist[u]= {key:val for key, val in shortest_path.items()} #distances in MIIA
        MIA_prev_nodes[u]= [(key,val) for key, val in previous_nodes.items()] #skeleton of MIIA
        
        distances={}
        for v in S_N:
            if v in shortest_path.keys() and v in MIIA.nodes():
                distances[v]=shortest_path[v] # shortest distance from v to u
        D[u]=distances #distances to negatively affected node u from negative seed
        
        D_max[u]=max([d for k,d in D[u].items()],default=0) # longest distance from a S_N to u
        print(D_max[u])
        subset_nodes=[v for v,l in dist[u].items() if l > D_max[u]] #nodes whose distance in the MIIA of u is smaller or equal to d_max
        MIIA_sub=MIIA.copy()
        MIIA_sub.remove_nodes_from(subset_nodes)
        ap=compute_ap_N(u,S_N,MIIA_sub,G)
        
        PIIS=compute_PIIS(u,G,D_max[u]) # nodes that closer to u then the furthest negative seed
        for v in PIIS:
            prov_S_P= S_P + [v]
            dc_v= find_dc(G,prov_S_P,u) # v is the only node in S_P so far
            dmax_v=max([d for k,d in D[u].items() if d<dc_v],default=0)
            
            if dmax_v ==0:
                ap_v=0
            else:
                
                subset_nodes_v=[v for v,l in shortest_path.items() if sys.maxsize> l > dmax_v]
                MIIA_sub_v=MIIA.copy()
                MIIA_sub_v.remove_nodes_from(subset_nodes_v)
                ap_v=compute_ap_N(u,S_N,MIIA_sub_v,G)
            
            if v in DecInf:
                DecInf[v]+=(ap-ap_v)
            else:
                DecInf[v]=(ap-ap_v) #compute total impact of the node as truth campaigner
                
    for i in range(k): 
        candidates={k:v for k,v in DecInf.items() if k not in S_N+list(S_P)}
        u = max(candidates,key=candidates.get) # chose node with higest impact
        PIOS=compute_PIOS(u,G,D_max) 
        
        for v in PIOS: # nodes that u could potentially save
            for w in compute_PIIS(v,G,D_max[v]): #nodes whose contribution now decreses since u helps
                prov_S_P= S_P + [w]
                if v  not in Neg_range: # this nodes are not affected by anything
                    continue
                
                dc_w= find_dc(G,S_P,v)
                dmax_w=max([d for k,d in D[v].items() if d<dc_w],default=0) 
                
                MIIA.clear()
                nodes={t[0] for t in MIA_prev_nodes[v]}
                nodes.add(v)
                MIIA.add_nodes_from(list(nodes))
                MIIA.add_edges_from(MIA_prev_nodes[v])
                
                subset_nodes=[k for k,d in D[v].items() if d > D_max[v]]
                MIIA_sub=MIIA.copy()
                MIIA_sub.remove_nodes_from(subset_nodes)
                ap=compute_ap_N(v,S_N,MIIA_sub,G)
                
                subset_nodes_w=[k for k,d in D[v].items() if d > dmax_w]
                MIIA_sub_w=MIIA.copy()
                MIIA_sub_w.remove_nodes_from(subset_nodes_w)
                ap_w=compute_ap_N(v,S_N,MIIA_sub_w,G) 
                if w in DecInf:
                    DecInf[w]-=(ap-ap_w) #remove all influence of w when u was not in S_P 
       
        S_P.append(u)
    
        for v in list(set(PIOS) - set([u])):
            if v not in Neg_range:
                continue
            dc=find_dc(G,S_P,v)
            D_max[v]= max([d for k,d in D[v].items() if d<dc],default=0)
            
            MIIA.clear()
            nodes={t[0] for t in MIA_prev_nodes[v]}
            nodes.add(v)
            MIIA.add_nodes_from(list(nodes))
            MIIA.add_edges_from(MIA_prev_nodes[v])

            subset_nodes=[k for k,d in D[v].items() if d > D_max[v]]
            MIIA_sub=MIIA.copy()
            MIIA_sub.remove_nodes_from(subset_nodes)
            ap=compute_ap_N(v,S_N,MIIA_sub,G)
            
                            
            for w in compute_PIIS(v,G,D_max[v]):
                prov_S_P= S_P + [w]
                dc_w= find_dc(G,prov_S_P,v)
                dmax_w=max([d for k,d in D[v].items() if d<dc_w],default=0)
                
                if dmax_w ==0:
                    ap_w=0
                else:
                    subset_nodes_k=[k for k,d in D[v].items() if d > dmax_w]
                    MIIA_sub_w=MIIA.copy()
                    MIIA_sub_w.remove_nodes_from(subset_nodes_w)
                    ap_w=compute_ap_N(v,S_N,MIIA_sub_w,G)
                if w in DecInf:
                    DecInf[w]+=(ap-ap_w)  
    stop = timeit.default_timer()
    print('Time: ', stop - start)          
                    
    return S_P 
k_n=5
S_N=[t[0] for t in sorted(Graph.degree, key=lambda x: x[1], reverse=True)][0:k_n]
print(S_N)
print(CMIA_H(Graph,S_N,10,0.2))    

In [ ]:
def get_communities(G):
    communities = []
    for i in G.nodes():
        if G.nodes[i]["community"] not in communities:
            communities.append(G.nodes[i]["community"])
    return len(communities)

In [ ]:
def CMIA_H_fair_version(G,S_N,k,min_prob): # for this algo we assume the prob of possitive diffusion is 1
    num_communities= get_communities(G)
    DecInf_percommunity=[{}]*num_communities
    print(DecInf_percommunity)
    return "hi"
    S_P=[]
    Neg_range=set()
    DecInf={}
    dist={}
    D={} # dict to keep track of the distances values per node, and avoid reconstructing MIA
    D_max=[0]*len(G.nodes()) # dict to keep track of the dmax values per node
    MIA_prev_nodes={}# to keep track of the MIIAs without having to reconstruct
    
    start = timeit.default_timer()

    for u in S_N:
        MIOA,_,_=compute_MIA(u, G, False, min_prob)
        for v in list(set(MIOA.nodes()) - set(S_N)):
            Neg_range.add(v) # contains the nodes that could get infected
    for u in Neg_range: 
        MIIA,shortest_path,previous_nodes=compute_MIA(u, G, True, min_prob)
        
        dist[u]= {key:val for key, val in shortest_path.items()} #distances in MIIA
        MIA_prev_nodes[u]= [(key,val) for key, val in previous_nodes.items()] #skeleton of MIIA
        
        distances={}
        for v in S_N:
            if v in shortest_path.keys() and v in MIIA.nodes():
                distances[v]=shortest_path[v] # shortest distance from v to u
        D[u]=distances #distances to negatively affected node u from negative seed
        
        D_max[u]=max([d for k,d in D[u].items()],default=0) # longest distance from a S_N to u
        
        subset_nodes=[v for v,l in dist[u].items() if l > D_max[u]] #nodes whose distance in the MIIA of u is smaller or equal to d_max
        MIIA_sub=MIIA.copy()
        MIIA_sub.remove_nodes_from(subset_nodes)
        ap=compute_ap_N(u,S_N,MIIA_sub,G)
        
        PIIS=compute_PIIS(u,G,D_max[u]) # nodes that closer to u then the furthest negative seed
        
        for v in PIIS:
                
            prov_S_P= S_P + [v]
            dc_v= find_dc(G,prov_S_P,u) # v is the only node in S_P so far
            dmax_v=max([d for k,d in D[u].items() if d<dc_v],default=0)
           
            
            if dmax_v ==0:
                ap_v=0
            else:
                subset_nodes_v=[v for v,l in shortest_path.items() if sys.maxsize> l > dmax_v]
                MIIA_sub_v=MIIA.copy()
                MIIA_sub_v.remove_nodes_from(subset_nodes_v)
                ap_v=compute_ap_N(u,S_N,MIIA_sub_v,G)
                
            if v in DecInf:
                DecInf[v]+=(ap-ap_v)
            else:
                DecInf[v]=(ap-ap_v) #compute total impact of the node as truth campaigner  
    for i in range(k): 
        candidates={k:v for k,v in DecInf.items() if k not in S_N+list(S_P)}
        if i==0:
            u = max(candidates,key=candidates.get) # chose node with higest overall impact
        else:
           "hi" 
        
        PIOS=compute_PIOS(u,G,D_max) 
        for v in PIOS: # nodes that u could potentially save
            for w in compute_PIIS(v,G,D_max[v]): #nodes whose contribution now decreses since u helps
                prov_S_P= S_P + [w]
                if v  not in Neg_range: # this nodes are not affected by anything
                    continue
                 
                dc_w= find_dc(G,S_P,v)
                dmax_w=max([d for k,d in D[v].items() if d<dc_w],default=0) 
                
                MIIA.clear()
                nodes={t[0] for t in MIA_prev_nodes[v]}
                nodes.add(v)
                MIIA.add_nodes_from(list(nodes))
                MIIA.add_edges_from(MIA_prev_nodes[v])
                
                subset_nodes=[k for k,d in D[v].items() if d > D_max[v]]
                MIIA_sub=MIIA.copy()
                MIIA_sub.remove_nodes_from(subset_nodes)
                ap=compute_ap_N(v,S_N,MIIA_sub,G)
                
                subset_nodes_w=[k for k,d in D[v].items() if d > dmax_w]
                MIIA_sub_w=MIIA.copy()
                MIIA_sub_w.remove_nodes_from(subset_nodes_w)
                ap_w=compute_ap_N(v,S_N,MIIA_sub_w,G) 
                
                DecInf[w]-=(ap-ap_w) #remove all influence of w when u was not in S_P 
       
        S_P.append(u)
    
        for v in list(set(PIOS) - set([u])):
            if v not in Neg_range:
                continue
            dc=find_dc(G,S_P,v)
            D_max[v]= max([d for k,d in D[v].items() if d<dc],default=0)
            
            MIIA.clear()
            nodes={t[0] for t in MIA_prev_nodes[v]}
            nodes.add(v)
            MIIA.add_nodes_from(list(nodes))
            MIIA.add_edges_from(MIA_prev_nodes[v])

            subset_nodes=[k for k,d in D[v].items() if d > D_max[v]]
            MIIA_sub=MIIA.copy()
            MIIA_sub.remove_nodes_from(subset_nodes)
            ap=compute_ap_N(v,S_N,MIIA_sub,G)
            
                            
            for w in compute_PIIS(v,G,D_max[v]):
                prov_S_P= S_P + [w]
                dc_w= find_dc(G,prov_S_P,v)
                dmax_w=max([d for k,d in D[v].items() if d<dc_w],default=0)
                
                if dmax_w ==0:
                    ap_w=0
                else:
                    subset_nodes_k=[k for k,d in D[v].items() if d > dmax_w]
                    MIIA_sub_w=MIIA.copy()
                    MIIA_sub_w.remove_nodes_from(subset_nodes_w)
                    ap_w=compute_ap_N(v,S_N,MIIA_sub_w,G)

                DecInf[w]+=(ap-ap_w)
          
    stop = timeit.default_timer()
    print('Time: ', stop - start)          
                    
    return S_P           
print(CMIA_H_fair_version(Graph,[1,2,3],4,0.1))    

In [ ]:
#functions used in the MCICM simulation
def set_all_Neu(G): # function to seet all nodes to neutral
    for i in G.nodes():
        G.nodes[i]['action'] = 'Neu'
    return G   

def set_Neg(G, list1): # function to set nodes in the negative seed to negative and active (Neg_a)
    for i in list1:
        G.nodes[i]['action'] = 'Neg_a'
    return G   

def set_Pos(G, list1): # function to set nodes in the positive seed to positive and active (Pos_a)
    for i in list1:
        G.nodes[i]['action'] = 'Pos_a'
    return G 
def get_colors(G): #functions to translate state to color for visualization
    color = []
    for i in G.nodes():
        if (G.nodes[i]['action'] == 'Neg' or G.nodes[i]['action'] == 'Neg_a'):
            color.append('red')
        elif (G.nodes[i]['action'] == 'Pos'or G.nodes[i]['action'] == 'Pos_a'):
            color.append('green')
        else:
            color.append('blue')
    return color

def recalculate(G): # function to update the state of each node after a step
    dict1 = {}
    negative=[]
    positive=[]
    
    for i in G.nodes():# we store all those nodes that are active to iterate over them
        if (G.nodes[i]['action'] == 'Neg_a'): 
            negative.append(i)
        if (G.nodes[i]['action'] == 'Pos_a'):
            positive.append(i)
    for i in positive: # the positive nodes have preference
        neigh = G.neighbors(i)
        for j in neigh:
            if (G.nodes[j]['action'] == 'Neu'): # if node has no opinion yet the positive influence always propagates
                G.nodes[j]['action'] = 'Pos_a'
        G.nodes[i]['action'] = 'Pos' # and the node is no longer active
        
    for i in negative:
        neigh = G.neighbors(i)
        for j in neigh:
            if (G.nodes[j]['action'] == 'Neu' and random.uniform(0, 1)<(1/G.in_degree(j))):
                G.nodes[j]['action'] = 'Neg_a'
        G.nodes[i]['action'] = 'Neg' # and the node is no longer active
    return G
                
def Calculate(G,n): # function that given a graph G with nodes in different states runs a MCICM for n iterations
    colors = get_colors(G)
    neg_seed_size=colors.count('red')
    pos_seed_size=colors.count('green')
    terminate = True
    count = 0
    while (terminate and count < n):
        count += 1
        G=recalculate(G)
        colors = get_colors(G)
        if (colors.count('red') == len(colors)- pos_seed_size or colors.count('green') == len(colors)-  neg_seed_size): # if all nodes are infected or cured we stop
            terminate = False
    return G

In [ ]:
def MCICM(G,S_N,S_P):
    G1 = G.copy()
    G1_start=set_Neg(set_all_Neu(G1),S_N)
    colors = get_colors(G1_start)
    pos = nx.kamada_kawai_layout(G1_start)
    nx.draw(G1_start, pos,with_labels=1, node_color=get_colors(G1), node_size=800)
    plt.show()
    
    print(S_P)
    
    G1=set_Pos(set_Neg(set_all_Neu(G1),S_N),S_P)
    colors = get_colors(G1)
    pos = nx.kamada_kawai_layout(G1)
    nx.draw(G1, pos,with_labels=1, node_color=get_colors(G1), node_size=800)
    plt.show()
    G1=Calculate(G1,100)
    colors = get_colors(G1)
    pos = nx.kamada_kawai_layout(G1)
    nx.draw(G1, pos,with_labels=1, node_color=get_colors(G1), node_size=800)
    plt.show()
    print(colors.count('red'))
    
    degrees=sorted(G.degree, key=lambda x: x[1], reverse=True)
    candidates= list(set([t[0] for t in degrees])-set(S_N))
    S_P_2= candidates[0:len(S_P)]
    print(S_P_2)
    G2 = G.copy()
    G2=set_Pos(set_Neg(set_all_Neu(G2),S_N), S_P_2)
    pos = nx.kamada_kawai_layout(G2)
    colors = get_colors(G2)
    nx.draw(G2,pos, with_labels=1, node_color=colors, node_size=800)
    plt.show()
    
    G2=Calculate(G2,100)
    colors = get_colors(G2)
    pos = nx.kamada_kawai_layout(G2)
    nx.draw(G2,pos, with_labels=1, node_color=colors, node_size=800)
    plt.show()
    print(colors.count('red'))
    

print(len(Graph.nodes()))
k_n=7
S_N=[t[0] for t in sorted(Graph.degree, key=lambda x: x[1], reverse=True)][0:k_n]

print(S_N)
MCICM(Graph,S_N,CMIA_H(Graph,S_N,5,0))